In [142]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [143]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [403]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [404]:
%%capture pwd
!pwd

In [405]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-07-01-how-are-developing-economies-coping-with-the-covid-19-crisis',
 300,
 500)

# Fig 1

In [406]:
df = pd.read_excel(
    "raw/gfsr-apr-2020-ch1.xlsx", sheet_name="Figure 1.7.", usecols="J:L", skiprows=8
).dropna(how="all")
df.columns = ["country", "through", "drawdown"]

In [407]:
clink = [
    "Poland",
    "Hungary",
    "Turkey",
    "China",
    "India",
    "Romania",
    "Bulgaria",
    "Philippines",
]
df = df.join(
    pd.DataFrame(
        {i: "no" if i in clink else "yes" for i in df["country"].unique()},
        index=["clink"],
    ).T,
    on="country",
)
df = df[~df["country"].isin(["Argentina", "Czech Rep."])]

In [408]:
f = "fig1_drawdown"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,country,through,drawdown,clink
0,China,-1.128055,-2.088991,no
1,Philippines,0.000000,-2.173252,no
2,Bulgaria,-2.529217,-4.657248,no
3,Romania,-3.458582,-5.849701,no
4,India,-6.182777,-6.994800,no


In [409]:
base = (
    (
        alt.Chart(f1)
        .encode(
            tooltip="tooltip:N",
            x=alt.X(
                "x:Q",
                axis=alt.Axis(
                    grid=False,
                    title="",
                    labelColor=colors["eco-gray"],
                    titleColor=colors["eco-gray"],
                    tickColor=colors["eco-gray"],
                    domainColor=colors["eco-gray"],
                    format=".0%",
                    tickCount=10,
                    orient="bottom",
                    labelAngle=0,
                ),
            ),
            y=alt.Y(
                "country:N",
                sort=[],
                axis=alt.Axis(
                    grid=False,
                    title="",
                    titleX=0,
                    titleY=-5,
                    titleBaseline="bottom",
                    titleAngle=0,
                    titleAlign="left",
                    orient="right",
                    labelColor=colors["eco-gray"],
                    titleColor=colors["eco-gray"],
                    tickColor=colors["eco-gray"],
                    domainColor=colors["eco-gray"],
                ),
            ),
        )
        .transform_calculate(
            tooltip="datum.country+' 🔽 '+round(datum.drawdown*10)/10.0+'%'+' 4️⃣ '+round(datum.through*10)/10.0+'%'"
        )
    )
    .transform_calculate(x="datum.drawdown/100")
    .transform_calculate(p="datum.through/100")
)
bars = base.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"]).encode(
    color=alt.Color(
        "clink:N",
        legend=None,
        scale=alt.Scale(range=[colors["eco-turquiose"], colors["eco-mid-blue"]]),
    )
)
points = base.mark_circle(size=80, color=colors["eco-yellow"], opacity=1).encode(
    x="p:Q"
)
text = bars.mark_text(dx=3, dy=1, align="left").encode(
    text=alt.Text("x:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
labels1 = (
    base.mark_text(dx=-3, dy=1, align="right", color=colors["eco-mid-blue"])
    .encode(text=alt.Text("l:N"))
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["Malaysia"]))
    .transform_calculate(l="'Commodity-linked'")
)
labels2 = (
    base.mark_text(dx=-3, dy=1, align="right", color=colors["eco-turquiose"])
    .encode(text=alt.Text("l:N"))
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["Poland"]))
    .transform_calculate(l="'Others'")
)
labels3 = (
    base.mark_text(
        dx=-3, dy=1, align="right", color=scale_lightness(colors["eco-yellow"], 0.8)
    )
    .encode(text=alt.Text("l:N"))
    .transform_filter(alt.FieldOneOfPredicate(field="country", oneOf=["Turkey"]))
    .transform_calculate(l="'Through April 9'")
)

layer1 = (
    (
        (bars + text + points + labels1 + labels2 + labels3).properties(
            height=alt.Step(18), width=300
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 2

In [410]:
df = pd.read_excel(
    "raw/Fig2.xlsx",
).dropna(how="all")
df = (
    df.set_index(["Sector", "Market", "Country"])
    .stack()
    .reset_index()
    .replace("Other Financial Institutions", "OFI")
)
df.columns = ["sector", "market", "country", "time", "quintile"]

In [411]:
f = "fig2_quintiles"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
df.head()

,sector,market,country,time,quintile
0,Sovereign,Advanced Economies,United States,Apr 2020,2
1,Sovereign,Advanced Economies,United States,Oct 2020,1
2,Sovereign,Advanced Economies,United States,Apr 2021,1
3,Companies,Advanced Economies,United States,Apr 2020,2
4,Companies,Advanced Economies,United States,Oct 2020,2


In [412]:
base = (
    alt.Chart(f2)
    .encode(tooltip="tooltip:N")
    .transform_calculate(
        tooltip="datum.country+' 🏢 '+datum.sector+' 📆 '+ datum.time+' 📊 quintile'+datum.quintile"
    )
)
heatmap = base.mark_rect(width=13, height=13).encode(
    x=alt.X(
        "time:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            orient="bottom",
            labelAngle=-45,
        ),
    ),
    y=alt.Y(
        "country:N",
        sort=[],
        axis=alt.Axis(
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "quintile:O",
        scale=alt.Scale(
            range=[
                colors["eco-pink"],
                scale_lightness(colors["eco-pink"], 1.6),
                scale_lightness(colors["eco-gray"], 1.7),
                scale_lightness(colors["eco-light-blue"], 1.3),
                scale_lightness(colors["eco-light-blue"], 0.8),
            ]
        ),
        legend=alt.Legend(
            title="Quintile",
            orient="top",
            titleOrient="left",
            titleFontWeight="normal",
            titleFontSize=10,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
        ),
    ),
    column=alt.Column(
        "sector:N", sort=[], title="", header=alt.Header(labelColor=colors["eco-gray"])
    ),
)
layer1 = (
    ((heatmap).properties(height=alt.Step(16), width=alt.Step(16)))
    .configure_view(stroke=None)
    .properties(title="")
)

layer1.save("visualisation/" + f + ".json")
layer1

alt.Chart(...)

In [399]:
base = (
    alt.Chart(f2)
    .encode(tooltip="tooltip:N")
    .transform_calculate(
        tooltip="datum.country+' 🏢 '+datum.sector+' 📆 '+ datum.time+' 📊 quintile'+datum.quintile"
    )
)
heatmap = base.mark_rect(width=13, height=13).encode(
    x=alt.X(
        "time:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            orient="bottom",
            labelAngle=-45,
        ),
    ),
    y=alt.Y(
        "country:O",
        sort=[],
        axis=alt.Axis(
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "quintile:O",
        scale=alt.Scale(
            range=[
                colors["eco-pink"],
                scale_lightness(colors["eco-pink"], 1.6),
                scale_lightness(colors["eco-gray"], 1.7),
                scale_lightness(colors["eco-light-blue"], 1.3),
                scale_lightness(colors["eco-light-blue"], 0.8),
            ]
        ),
        legend=alt.Legend(
            title="Quintile",
            orient="top",
            titleOrient="left",
            titleFontWeight="normal",
            titleFontSize=10,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
        ),
    ),
    column=alt.Column(
        "sector:N", sort=[], title="", header=alt.Header(labelColor=colors["eco-gray"])
    ),
    row=alt.Column(
        "market:N",
        title="",
        header=alt.Header(labelColor=colors["eco-gray"], labelAngle=0,labelAlign='left'),
    ),
)
layer1 = (
    ((heatmap).properties(height=alt.Step(16), width=alt.Step(16)))
    .configure_view(stroke=None)
    .properties(title="")
)

layer1.save("visualisation/" + f + ".json")
layer1

alt.Chart(...)